# 유튜브 댓글 웹크롤링 및 스크래핑

In [1]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

driver = webdriver.Chrome("/Users/yeonju1/Documents/bigData/src/07_Python/chromedriver")

items = ['인구감소']
count = 0
current = 0

for item in items :
    current += 1
    goal = len(items)
    print('진행시작')
    driver.get("https://www.google.com/") # 구글 열기
    searchbox = driver.find_element_by_name('q') # 검색창에 입력
    searchbox.send_keys(items)
    time.sleep(1)
    searchbox.submit()
    time.sleep(1)
    morebutton = driver.find_element_by_xpath("//div[@class='MXl0lf mtqGb']") # 동영상 더보기버튼 누름
    driver.execute_script("arguments[0].click();", morebutton)
    time.sleep(2)
    driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_lists = driver.find_elements_by_xpath("//a[@class='rGhul IHSDrd']")
    if len(review_lists) != 0 :
        id_final = []
        comment_final = []
        for i, review in enumerate(review_lists) :    
            print('검색결과 존재')
            print('진행상황 : ', i,'/',len(review_lists),sep="")
            reviewbutton = driver.find_elements_by_xpath("//a[@class='rGhul IHSDrd']")
            time.sleep(2)
            content_url = review_lists[i].get_attribute("href")
            tabs = driver.window_handles
            driver.switch_to_window(tabs[1]) # 새 탭으로 이동
            driver.get(content_url) # 링크 열기
            time.sleep(2)

            # 유튜브 크롤링 시작
            driver.execute_script("window.scrollTo(0, 800)")
            time.sleep(3)

            # 페이지 스크롤
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                time.sleep(1.5)

                new_height = driver.execute_script("return document.documentElement.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            time.sleep(1.5)

            # 팝업 닫기
            try:
                driver.find_element_by_css_selector("#dismiss-button > a").click()
            except:
                pass

            # html에서 댓글 추출ㄴ
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            id_list = soup.select("div#header-author > h3 > #author-text > span")
            comment_list = soup.select("yt-formatted-string#content-text")

            for i in range(len(comment_list)):
                temp_id = id_list[i].text
                temp_id = temp_id.replace('\n', '')
                temp_id = temp_id.replace('\t', '')
                temp_id = temp_id.replace('    ', '')
                id_final.append(temp_id)

                temp_comment = comment_list[i].text
                temp_comment = temp_comment.replace('\n', '')
                temp_comment = temp_comment.replace('\t', '')
                temp_comment = temp_comment.replace('    ', '')
                comment_final.append(temp_comment)

            driver.switch_to_window(tabs[0])
            print("검색 페이지로 돌아감")
else:
    print("더이상 검색결과 없음")
    
pd_data = {"ID" : id_final , "Comment" : comment_final}
youtube_pd = pd.DataFrame(pd_data)
youtube_pd
print("OK 크롤링 종료")
driver.quit()

진행시작
검색결과 존재
진행상황 : 0/10


<ipython-input-1-59925a6db3d5>:42: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) # 새 탭으로 이동
<ipython-input-1-59925a6db3d5>:88: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


검색 페이지로 돌아감
검색결과 존재
진행상황 : 1/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 2/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 3/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 4/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 5/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 6/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 7/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 8/10
검색 페이지로 돌아감
검색결과 존재
진행상황 : 9/10
검색 페이지로 돌아감
더이상 검색결과 없음
OK 크롤링 종료


In [8]:
youtube_pd.to_csv('youtube_crawling.csv', mode='w' , encoding='UTF8')
youtube_pd

,ID,Comment
0,개미와베짱이,대한민국에서 가장 높은 산: 부동산대한민국에서 가장 낮은 산: 출산
1,sm S,못막은게 아니라 안막은거지 ㅋㅋㅋ100조를 그따구로 쓰지말고출산인구에 나눠만 줬어도...
2,김이,애낳으면 몇천만원씩 준다던 공략이 제일 현실적이였음을 느낌진짜 재평가의 재평가다 ㅋ...
3,me coll,자국민들이 피땀흘려 이룩해놓은 나라인데 중국인들이 와서 혜택누리겠네 ㅋㅋ
4,무한한한계,??? : 그래서 플랜B는 뭐야???? : 플랜B 없어!??? : 아...답변 고마워
...,...,...
6522,h.s lee,마지막 한마디 더하자면 국방의 병력자원이 모자르면 외국의 저렴한 용병을 구매해서 최...
6523,낙타,남성여성 양극화시키고트젠들 옹호하고인구감소가 목적인듯 여가부 설치는걸 보고도 원인...
6524,코콩코코콩콩,애낳으면 국가가 책임지고 성인까지 성장하는데 드는 식비.의류비.교육비 모두 대줄건가...
6525,만두,일본한테 지기 싫으면 애많이 나아야 애국이다 이것드라 매국노가 아닌이상 애를 안낳을...


In [75]:
youtube_pd.isna().sum()

ID         0
Comment    0
dtype: int64